<a href="https://colab.research.google.com/github/dany-xu/AI-Generated-Text-Detection-using-LLM/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Sun Apr 28 18:55:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # current dir: '/content'
import pandas as pd
import torch
import random
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from transformers import get_scheduler
from transformers import pipeline

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install torchkeras

In [ ]:
data = pd.read_csv("./drive/MyDrive/ColabNotebooks/llm/concat_ori.csv").drop(columns=["Unnamed: 0"])
# data = data[data['abstract'].str.len() <= 2048]
data1 = data[data['label'] == 1]
data0 = data[data['label'] == 0]
train1, test1 = train_test_split(data1, test_size=0.2)
train0, test0 = train_test_split(data0, test_size=0.2)
train = pd.concat([train0, train1], ignore_index=True)
test = pd.concat([test0, test1], ignore_index=True)
train = train.loc[np.random.permutation(train.index)]
test = test.loc[np.random.permutation(test.index)]
train.shape, test.shape

((40559, 5), (10140, 5))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
train_data = tokenizer(list(train.abstract), padding = "max_length", max_length = 512, truncation=True ,return_tensors = "pt")
train_label = list(train.label)
test_data = tokenizer(list(test.abstract), padding = "max_length", max_length = 512, truncation=True ,return_tensors = "pt")
test_label = list(test.label)

In [ ]:
train_data

{'input_ids': tensor([[  101,  1037,  3811,  ...,     0,     0,     0],
        [  101,  7976,  4454,  ...,     0,     0,     0],
        [  101, 15544,  2078,  ...,     0,     0,     0],
        ...,
        [  101,  1999,  2023,  ...,     0,     0,     0],
        [  101,  3746,  3609,  ...,     0,     0,     0],
        [  101,  3891,  7667,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
batch_size = 128
trainset = TensorDataset(train_data["input_ids"], train_data["attention_mask"], torch.tensor(train_label))
train_sampler = RandomSampler(trainset)
train_dataloader = DataLoader(trainset, sampler=train_sampler, batch_size=batch_size)

NameError: name 'TensorDataset' is not defined

In [ ]:
train_data["input_ids"].shape

In [ ]:
'''
for batch in train_dataloader:
    input_ids, attention_mask, labels = batch
    # input_ids: Tensor of input IDs for the batch
    # attention_mask: Tensor of attention masks for the batch
    # labels: Tensor of labels for the batch

    # Perform operations using the batch, such as passing it through your model for training
    # For example:
    # optimizer.zero_grad()
    # outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    # loss = outputs.loss
    # loss.backward()
    # optimizer.step()

    # Print or process the batch as needed
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
    print(input_ids.shape, attention_mask.shape, labels.shape) # torch.Size([256, 2048]) torch.Size([256, 2048]) torch.Size([256])

In [ ]:
len(train_dataloader) # iterate num

2535

In [ ]:
# optimizer setting
optimizer = AdamW(model.parameters(), lr=1e-4)
# epoch
epoch_num = 1
num_training_steps = epoch_num * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
for epoch in range(epoch_num):
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            print("step: ",step, "  loss:",total_loss/(step*batch_size))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
    avg_train_loss = total_loss / len(train_dataloader)
    print("avg_loss:",avg_train_loss)


step:  10   loss: 0.04042543228715658
step:  20   loss: 0.03947176542133093
step:  30   loss: 0.03663086195786794
step:  40   loss: 0.03788367027882487
step:  50   loss: 0.035860747881233695
step:  60   loss: 0.033193931200852
step:  70   loss: 0.03197322979436389
step:  80   loss: 0.031149720383109526
step:  90   loss: 0.029940966873740156
step:  100   loss: 0.029353569322265686
step:  110   loss: 0.028706809984181413
step:  120   loss: 0.028108369159357
step:  130   loss: 0.0270018816101723
step:  140   loss: 0.026056477184673504
step:  150   loss: 0.02577480261757349
step:  160   loss: 0.025567557960312116
step:  170   loss: 0.025227295903845086
step:  180   loss: 0.024666826322855842
step:  190   loss: 0.0247629412933939
step:  200   loss: 0.024637984826113096
step:  210   loss: 0.024257805333140172
step:  220   loss: 0.024064814074451782
step:  230   loss: 0.023719095126422042
step:  240   loss: 0.02341592762725971
step:  250   loss: 0.023179466996341942
step:  260   loss: 0.02372

In [ ]:
# example test
'''
question = ""
#test_data = tokenizer(list(test.abstract), padding = "max_length", max_length = 512, truncation=True ,return_tensors = "pt")
#test_label = list(test.label)
test = tokenizer(inp,return_tensors="pt",padding="max_length",max_length=512)

model.eval()
with torch.no_grad():
    test["input_ids"] = test["input_ids"].to(device)
    test["attention_mask"] = test["attention_mask"].to(device)
    outputs = model(test["input_ids"],
                    token_type_ids=None,
                    attention_mask=test["attention_mask"])
pred_flat = np.argmax(outputs["logits"].cpu(),axis=1).numpy().squeeze()
pred_flat.tolist()

In [ ]:
model.eval()

In [ ]:
# save model
id2label_dic={}
for k,v in label_dic.items():
    id2label_dic[v] = k
id2label_dic[pred_flat.tolist()]
model.config.id2label = id2label_dic
model.save_pretrained("./drive/MyDrive/ColabNotebooks/llm/bert1")
tokenizer.save_pretrained("./drive/MyDrive/ColabNotebooks/llm/bert1")

In [ ]:
# call model
classifier = pipeline("text-classification",model="./drive/MyDrive/ColabNotebooks/llm/bert1")
result = []
for idx, d in test.iterrows():
    sentence = d[3][:512] # abstract
    pred_answer = classifier(sentence)
    print(idx, pred_answer)
    result.append({"abstract": d[3], "true_label": d[4], "pred_label": pred_answer}) # abstract, label


result = pd.DataFrame(result)
result.to_csv("./drive/MyDrive/ColabNotebooks/llm/bert_result.csv")

In [ ]:
# measure results
true_labels = result['true_label'].tolist()
pred_labels = result['pred_label'].tolist()
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)
conf_matrix = confusion_matrix(true_labels, pred_labels) # confusion matrix

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:", conf_matrix)
print("Among above, unrecognized sample numbers: ", (result['pred_label'] == 2).sum())